In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Graphics
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
import lightgbm as lgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import f1_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

# Divers
from time import time, strftime, gmtime
import gc
import pickle

# Module des fonctions du notebook
import fonctions08 as f

# Autoreload pour prise en compte des changments dans le module fonctions
%load_ext autoreload
%autoreload 1
%aimport fonctions08

In [2]:
# Heure démarrage
t0=time()

# Constantes
RANDOM_STATE = 1
NROWS = None
NCOLS = None
N_FOLDS = 5

# Paramètres pour les validations croisées
random_seed = 1 # Seed pour générateur de nombre aléatoire
folds = 5 # Nombre de folds pour validation croisée

In [3]:
features = f.import_csv("data/features/train_features_selected.csv", nrows=NROWS)
target = f.import_csv("data/features/train_target.csv", nrows=NROWS)

features = features.set_index("SK_ID_CURR")
target = target.set_index("SK_ID_CURR")

print(features.shape)
print(target.shape)

Memory usage of dataframe is 505.82 MB
Memory usage after optimization is: 164.84 MB
Decreased by 67.4%
Memory usage of dataframe is 3.28 MB
Memory usage after optimization is: 1.03 MB
Decreased by 68.7%
(215257, 307)
(215257, 1)


# Echantillon avec classes équilibrées

In [4]:
target[target['TARGET'] == 1].shape

(17377, 1)

In [5]:
target_0 = target[target['TARGET'] == 0]
target_1 = target[target['TARGET'] == 1]

target_0_sample = target_0.sample(len(target_1), random_state=RANDOM_STATE)

target_sample_balanced = pd.concat([target_0_sample, target_1], axis=0)
features_sample_balanced = features.loc[target_sample_balanced.index, :].iloc[:, :NCOLS]

In [6]:
features_sample_balanced.shape

(34754, 307)

In [7]:
# Creation d'un train set de 10000 observations et test set de 6000 obs
X_train, X_test, y_train, y_test =\
    train_test_split(features_sample_balanced, target_sample_balanced,
                     train_size=15000, test_size=6000,
                     stratify=target_sample_balanced, random_state=RANDOM_STATE)

# TEST CV + LGBM + F1 SCORE

In [8]:
import optuna

def objective(trial):
    params = {
        'verbosity':-1,
        'objective': 'binary',
        'metric': 'auc',        
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 20, 200),
        'num_leaves ': trial.suggest_int('num_leaves', 8, 128),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
    }
    
    k_fold = KFold(n_splits=N_FOLDS, shuffle=False, random_state=RANDOM_STATE)
    
    scores=[]

    for train_index, valid_index in k_fold.split(X_train):
    
        # Training data for the fold
        fold_X_train, fold_y_train = X_train.iloc[train_index, :], y_train.iloc[train_index, :]
        
        # Validation data for the fold
        fold_X_valid, fold_y_valid = X_train.iloc[valid_index, :], y_train.iloc[valid_index, :]
        
        model = lgb.LGBMModel(random_state=1)
        model.set_params(**params)
        model.fit(fold_X_train, np.ravel(fold_y_train))
        preds = model.predict(fold_X_valid)
        scores.append(f1_score(fold_y_valid, np.rint(preds)))
   
    return np.mean(scores)

In [9]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, timeout=None) # timeout (seconds)

[I 2020-01-03 21:50:06,034] Finished trial#0 resulted in value: 0.6991599437616148. Current best value is 0.6991599437616148 with parameters: {'n_estimators': 60, 'num_leaves': 22, 'max_depth': 8, 'learning_rate': 0.0929251127981718, 'min_child_samples': 100, 'colsample_bytree': 0.5563150103878447, 'reg_alpha': 0.8546680037284978}.
[I 2020-01-03 21:50:16,997] Finished trial#1 resulted in value: 0.6977214472432577. Current best value is 0.6991599437616148 with parameters: {'n_estimators': 60, 'num_leaves': 22, 'max_depth': 8, 'learning_rate': 0.0929251127981718, 'min_child_samples': 100, 'colsample_bytree': 0.5563150103878447, 'reg_alpha': 0.8546680037284978}.
[I 2020-01-03 21:50:30,464] Finished trial#2 resulted in value: 0.6927301989421221. Current best value is 0.6991599437616148 with parameters: {'n_estimators': 60, 'num_leaves': 22, 'max_depth': 8, 'learning_rate': 0.0929251127981718, 'min_child_samples': 100, 'colsample_bytree': 0.5563150103878447, 'reg_alpha': 0.8546680037284978}

[I 2020-01-03 21:54:37,624] Finished trial#24 resulted in value: 0.7027996835405375. Current best value is 0.7047007417920521 with parameters: {'n_estimators': 151, 'num_leaves': 85, 'max_depth': 5, 'learning_rate': 0.06397744929703832, 'min_child_samples': 46, 'colsample_bytree': 0.6121398411968751, 'reg_alpha': 0.5463885145955139}.
[I 2020-01-03 21:54:56,682] Finished trial#25 resulted in value: 0.7025634032361303. Current best value is 0.7047007417920521 with parameters: {'n_estimators': 151, 'num_leaves': 85, 'max_depth': 5, 'learning_rate': 0.06397744929703832, 'min_child_samples': 46, 'colsample_bytree': 0.6121398411968751, 'reg_alpha': 0.5463885145955139}.
[I 2020-01-03 21:55:02,481] Finished trial#26 resulted in value: 0.6955757379509429. Current best value is 0.7047007417920521 with parameters: {'n_estimators': 151, 'num_leaves': 85, 'max_depth': 5, 'learning_rate': 0.06397744929703832, 'min_child_samples': 46, 'colsample_bytree': 0.6121398411968751, 'reg_alpha': 0.54638851459

[I 2020-01-03 21:58:30,477] Finished trial#48 resulted in value: 0.7031193224938905. Current best value is 0.706640072580626 with parameters: {'n_estimators': 200, 'num_leaves': 84, 'max_depth': 5, 'learning_rate': 0.05374717380171641, 'min_child_samples': 100, 'colsample_bytree': 0.5029578787000597, 'reg_alpha': 0.5769233423982185}.
[I 2020-01-03 21:58:37,538] Finished trial#49 resulted in value: 0.7006651522079175. Current best value is 0.706640072580626 with parameters: {'n_estimators': 200, 'num_leaves': 84, 'max_depth': 5, 'learning_rate': 0.05374717380171641, 'min_child_samples': 100, 'colsample_bytree': 0.5029578787000597, 'reg_alpha': 0.5769233423982185}.
[I 2020-01-03 21:58:47,584] Finished trial#50 resulted in value: 0.7026332667087787. Current best value is 0.706640072580626 with parameters: {'n_estimators': 200, 'num_leaves': 84, 'max_depth': 5, 'learning_rate': 0.05374717380171641, 'min_child_samples': 100, 'colsample_bytree': 0.5029578787000597, 'reg_alpha': 0.57692334239

[I 2020-01-03 22:02:12,247] Finished trial#72 resulted in value: 0.7025092170531181. Current best value is 0.706640072580626 with parameters: {'n_estimators': 200, 'num_leaves': 84, 'max_depth': 5, 'learning_rate': 0.05374717380171641, 'min_child_samples': 100, 'colsample_bytree': 0.5029578787000597, 'reg_alpha': 0.5769233423982185}.
[I 2020-01-03 22:02:22,260] Finished trial#73 resulted in value: 0.7021164238992398. Current best value is 0.706640072580626 with parameters: {'n_estimators': 200, 'num_leaves': 84, 'max_depth': 5, 'learning_rate': 0.05374717380171641, 'min_child_samples': 100, 'colsample_bytree': 0.5029578787000597, 'reg_alpha': 0.5769233423982185}.
[I 2020-01-03 22:02:30,662] Finished trial#74 resulted in value: 0.7025101811045131. Current best value is 0.706640072580626 with parameters: {'n_estimators': 200, 'num_leaves': 84, 'max_depth': 5, 'learning_rate': 0.05374717380171641, 'min_child_samples': 100, 'colsample_bytree': 0.5029578787000597, 'reg_alpha': 0.57692334239

[I 2020-01-03 22:05:28,767] Finished trial#96 resulted in value: 0.704263994911011. Current best value is 0.706640072580626 with parameters: {'n_estimators': 200, 'num_leaves': 84, 'max_depth': 5, 'learning_rate': 0.05374717380171641, 'min_child_samples': 100, 'colsample_bytree': 0.5029578787000597, 'reg_alpha': 0.5769233423982185}.
[I 2020-01-03 22:05:35,271] Finished trial#97 resulted in value: 0.7040459980025491. Current best value is 0.706640072580626 with parameters: {'n_estimators': 200, 'num_leaves': 84, 'max_depth': 5, 'learning_rate': 0.05374717380171641, 'min_child_samples': 100, 'colsample_bytree': 0.5029578787000597, 'reg_alpha': 0.5769233423982185}.
[I 2020-01-03 22:05:40,303] Finished trial#98 resulted in value: 0.6922500250705955. Current best value is 0.706640072580626 with parameters: {'n_estimators': 200, 'num_leaves': 84, 'max_depth': 5, 'learning_rate': 0.05374717380171641, 'min_child_samples': 100, 'colsample_bytree': 0.5029578787000597, 'reg_alpha': 0.576923342398

...

[I 2020-01-04 01:03:40,615] Finished trial#888 resulted in value: 0.7059405925205338. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.754759977412128}.
[I 2020-01-04 01:03:57,861] Finished trial#889 resulted in value: 0.7060464938896438. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.754759977412128}.
[I 2020-01-04 01:04:14,785] Finished trial#890 resulted in value: 0.7059881955116117. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.7547599

[I 2020-01-04 01:10:17,774] Finished trial#912 resulted in value: 0.7034162682266596. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.754759977412128}.
[I 2020-01-04 01:10:34,076] Finished trial#913 resulted in value: 0.7070846662979593. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.754759977412128}.
[I 2020-01-04 01:10:48,088] Finished trial#914 resulted in value: 0.7050739854380641. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.7547599

[I 2020-01-04 01:16:11,861] Finished trial#936 resulted in value: 0.7046626732192669. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.754759977412128}.
[I 2020-01-04 01:16:28,169] Finished trial#937 resulted in value: 0.7046860966952003. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.754759977412128}.
[I 2020-01-04 01:16:42,317] Finished trial#938 resulted in value: 0.702017358687802. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.75475997

[I 2020-01-04 01:22:03,895] Finished trial#960 resulted in value: 0.7057730312135183. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.754759977412128}.
[I 2020-01-04 01:22:17,106] Finished trial#961 resulted in value: 0.7055636936027609. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.754759977412128}.
[I 2020-01-04 01:22:31,614] Finished trial#962 resulted in value: 0.7047576849747023. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.7547599

[I 2020-01-04 01:27:48,018] Finished trial#984 resulted in value: 0.7050365044554073. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.754759977412128}.
[I 2020-01-04 01:28:03,348] Finished trial#985 resulted in value: 0.7012096746886287. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.754759977412128}.
[I 2020-01-04 01:28:16,988] Finished trial#986 resulted in value: 0.7027456604778736. Current best value is 0.7086903989976203 with parameters: {'n_estimators': 195, 'num_leaves': 97, 'max_depth': 9, 'learning_rate': 0.054090817210931126, 'min_child_samples': 64, 'colsample_bytree': 0.6265551095821733, 'reg_alpha': 0.7547599

In [10]:
print('Number of finished trials: {}'.format(len(study.trials)))

print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 1000
Best trial:
  Value: 0.7086903989976203
  Params: 
    n_estimators: 195
    num_leaves: 97
    max_depth: 9
    learning_rate: 0.054090817210931126
    min_child_samples: 64
    colsample_bytree: 0.6265551095821733
    reg_alpha: 0.754759977412128


In [11]:
# Modele final
model = lgb.LGBMModel(objective='binary', metric='auc',
                      boosting_type= 'gbdt')
model.set_params(**study.best_params)
model

LGBMModel(boosting_type='gbdt', class_weight=None,
     colsample_bytree=0.6265551095821733, importance_type='split',
     learning_rate=0.054090817210931126, max_depth=9, metric='auc',
     min_child_samples=64, min_child_weight=0.001, min_split_gain=0.0,
     n_estimators=195, n_jobs=-1, num_leaves=97, objective='binary',
     random_state=None, reg_alpha=0.754759977412128, reg_lambda=0.0,
     silent=True, subsample=1.0, subsample_for_bin=200000,
     subsample_freq=0)

In [12]:
# On sauvegarde le modèle et la liste des features
model_params = {'model': model.get_params(), 'study': study}
filename = 'data/model/params_f1.sav'
pickle.dump(model_params, open(filename, 'wb'))

In [13]:
# Aperçu du dataframe des combinaisons de valeurs testées
df_res = pickle.load(open(filename, 'rb'))['study'].trials_dataframe()
df_res.head()

number                state     value             datetime_start  \
                                                                     
0      0  TrialState.COMPLETE  0.699160 2020-01-03 21:50:00.112691   
1      1  TrialState.COMPLETE  0.697721 2020-01-03 21:50:06.034663   
2      2  TrialState.COMPLETE  0.692730 2020-01-03 21:50:16.997342   
3      3  TrialState.COMPLETE  0.702906 2020-01-03 21:50:30.464533   
4      4  TrialState.COMPLETE  0.700007 2020-01-03 21:50:40.723797   

           datetime_complete           params                          \
                             colsample_bytree learning_rate max_depth   
0 2020-01-03 21:50:06.034663         0.556315      0.092925         8   
1 2020-01-03 21:50:16.997342         0.904244      0.050298         7   
2 2020-01-03 21:50:30.464533         0.721635      0.015387         7   
3 2020-01-03 21:50:40.723797         0.640332      0.075108         8   
4 2020-01-03 21:50:46.894217         0.781943      0.099998         6   

                                                      system_attrs  
  min_child_samples n_estimators num_leaves reg_alpha      _number  
0               100           60         22  0.854668            0  
1                20           69         37  0.379312            1  
2                70          125         17  0.238474            2  
3               100          122         46  0.988052            3  
4                56           58         72  0.619514            4

In [14]:
t1 = time()
print("computing time : {:8.6f} sec".format(t1-t0))
print("computing time : " + strftime('%H:%M:%S', gmtime(t1-t0)))

computing time : 13342.401447 sec
computing time : 03:42:22
